# 2301888682 - ICHSAN

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras import layers, Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [2]:
df = pd.DataFrame(columns = ["Data", "Label"],
                 data = [["I feel like I am drowning. #depression #anxiety #failure #worthless", "fear"],
                        ["#panic Panic attack from fear of starting new medication", "fear"],
                        ["My bus was in a car crash... I'm still shaking a bit... This week was an absolute horror and this was the icing on the cake... #terrible", "fear"],
                        ["Just got back from seeing @GaryDelaney in Burslem. AMAZING!! Face still hurts from laughing so much #hilarious", "joy"],
                        ["It's the #FirstDayofFall and I'm so happy. Sipping my #PumpkinSpice flavoured coffee and #smiling! Happy Fall everyone! #amwriting", "joy"],
                        ["Morning all! Of course it is sunny on this Monday morning to cheerfully welcome us back to work.:)", "joy"]])

df

,Data,Label
0,I feel like I am drowning. #depression #anxiet...,fear
1,#panic Panic attack from fear of starting new ...,fear
2,My bus was in a car crash... I'm still shaking...,fear
3,Just got back from seeing @GaryDelaney in Burs...,joy
4,It's the #FirstDayofFall and I'm so happy. Sip...,joy
5,Morning all! Of course it is sunny on this Mon...,joy


# A. Data Preprocessing
# a) Preprocess Sentence (X)
## 1. Tokenize menggunakan .split() 
### - Tujuan: agar dapat membuat sentence menjadi per word token.
## 2. Lowercase menggunakan .lower() 
### - Dalam sentiment analysis, melihat kata yang digunakan saja sudah cukup. 
### - Contoh: "AMAZING!" dan "amazing!", tidak peduli huruf besar atau kecil, yang penting keyword "amazing" nya lah yang membuatnya tergolong "joy".
## 3. Remove Stop Words 
### - Yang memiliki arti sentiment hanya keyword yang bukan stop words. 
### - Contoh: "am", "was" kita tidak bisa tahu mana "joy" atau "fear", kita hanya butuh keyword yang bukan stopword seperti "amazing", "horror".
## 4. Padding Cutting 
### - Setiap sentence memiliki panjang yang berbeda, maka perlu disamakan semuanya. 
### - Semua sentence akan dibuat sama panjang yaitu average dari panjang semua kalimat atau 10. 
### - Alasan mengambil panjang average: kalau mengambil panjang max, data traning sangat sedikit dan kebanyakan sentence label "fear" yang pendek akan mendapat banyak padding. Hal ini akan merusak model karena model akan menganggap sentence  yang banyak padding cenderung kategori "fear".
### - Padding dengan menambahkan ("").
## Alasan saya tidak menggunakan stemming adalah ketika saya melakukan stemming dengan bantuan nltk.stem PorterStemmer, hasilnya kurang baik seperti "depression" menjadi "depressi".
# b) Preprocess Label (Y)
## 1. Label hanya berupa binary classification ("fear", "joy"), jadi OrdinalEncoder sudah cukup karena bisa direpresentasikan oleh 0 dan 1.

In [3]:
def pad_cut(data, max_length):
    for i, sentence in enumerate(data):
        length = len(sentence)
        # padding
        if length < max_length:
            for j in range(length, max_length): data[i].append("")
        # cutting
        else:
            data[i] = data[i][:max_length]
    return data

def preprocess(data):
    MAX_LENGTH = 10
    # Remove Stop Words, Lowercase, Tokenize
    data = [remove_stopwords(sentence.lower()).split() for sentence in data]
    # Padding or Cutting into MAX_LENGTH
    data = pad_cut(data, MAX_LENGTH)
    return data

x = preprocess(df["Data"])

y = df[["Label"]]
scaler_y = OrdinalEncoder().fit(y)
y = scaler_y.transform(y)

x, y

([['feel',
   'like',
   'drowning.',
   '#depression',
   '#anxiety',
   '#failure',
   '#worthless',
   '',
   '',
   ''],
  ['#panic',
   'panic',
   'attack',
   'fear',
   'starting',
   'new',
   'medication',
   '',
   '',
   ''],
  ['bus',
   'car',
   'crash...',
   "i'm",
   'shaking',
   'bit...',
   'week',
   'absolute',
   'horror',
   'icing'],
  ['got',
   'seeing',
   '@garydelaney',
   'burslem.',
   'amazing!!',
   'face',
   'hurts',
   'laughing',
   '#hilarious',
   ''],
  ["it's",
   '#firstdayoffall',
   "i'm",
   'happy.',
   'sipping',
   '#pumpkinspice',
   'flavoured',
   'coffee',
   '#smiling!',
   'happy'],
  ['morning',
   'all!',
   'course',
   'sunny',
   'monday',
   'morning',
   'cheerfully',
   'welcome',
   'work.:)',
   '']],
 array([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]]))

# B. Word Embedding as Feature, number of hashtag as Feature
# a) Word Embedding as Feature
## 1. Cara kerja Model Word2Vec Gensim
### - Cara kerja word2vec adalah dengan membangun neural network (Input - Hidden - Output).
### - By default, word2vec dari gensim adalah CBOW. (Referensi: https://groups.google.com/g/gensim/c/neDSgepSpCk?pli=1)
### - Konsep CBOW adalah melatih neural network dengan: Input (pair word bersebelahan dengan target word sejauh window_size) dan Output (target word).
### - Contoh:
#### window_size = 1, sentence = "I like you bro"
#### Maka Training berupa:
#### "I like", Target Word = "I", sisanya pair word input (bersebelahan sejauh windowsize).
#### Input: (I, like), Output = (I)
#### "I like you", Target Word = "like", sisanya pair wod input (bersebelahan sejauh windowsize).
#### Input: (like, I), Output = (like)
#### Input: (like, you), Output = (like)
#### "like you bro", Target Word = "you", sisanya pair wod input (bersebelahan sejauh windowsize).
#### Input: (you, like), Output = (you)
#### Input: (you, bro), Output = (you)
#### "you bro", Target Word = "bro", sisanya pair wod input (bersebelahan sejauh windowsize).
#### Input: (bro, you), Output = (bro)
### - Setelah melakukan training denga input, output diatas yang merupakan konsep CBOW, maka weight antara Input Layer menuju Hidden Layer akan dijadikan sebagai representasi vector untuk word tersebut. (Referensei: https://israelg99.github.io/2017-03-23-Word2Vec-Explained/)
### - Tujuan digunakan weight Input Layer menuju Hidden Layer adalah karena ketika proses training, nilai weight sudah beradaptasi untuk merepresentasi data corpus trainingnya.
### - Untuk cara kerja ANN hanya berupa update weight sesuai error antara prediksi model dan real value.
## 2. Mengapa menggunakan word2vec pada masalah ini?
### - Neural Network tidak mengenal huruf dan hanya mengenal angka sehingga Word2Vec dapat digunakan untuk representasi word dalam bentuk vector angka.
### - Dengan representasi vector angka word2vec dapat merepresentasikan word similarity.
### - Feature word similarity untuk model akan sangat bermanfaat seperti jika dalam satu sentence banyak similarity word yang dekat maka akan lebih jelas hasilnya mengarah ke label mana.
# b) Number of Hashtags as Feature
## 1. Mengapa menggunakan number of hashtags sebagai feature? 
### - Permintaan dari soal, jadi saya ikuti. 
### - Menurut saya, feature ini tidak akan membawa peningkatan performa karena tidak ada hubungan antara jumlah Hashtags dan "Fear" "Joy" (saya mencobanya dan benar feature ini tidak membawa peningkatan performa). 
### - Contoh: "#Happy" dan "#Sad", keduanya memiliki jumlah hashtags = 1, akan tetapi kita tidak dapat menilai mana joy dan fear hanya dari jumlah hashtag.

## Count number of hashtags on each Sentence

In [4]:
num_tag = []

for sentence in x:
    tag = 0
    for word in sentence:
        if "#" in word: tag += 1
    num_tag.append(tag)
    
num_tag

[4, 1, 0, 1, 3, 0]

## Embedding Layer 
### - Convert Word menjadi vector representation menggunakan word2vec
### - Reshape menjadi shape yang diterima BPNN dari shape (6 sentences, 10 words, 100 vector representation) menjadi shape (6, 10 word * 100 vector) = (6, 1000)
### - Append Number of hashtags sebagai feature (6, 1000 + 1 Number of hashtags) = (6, 1001)

In [5]:
def vectorize(data):
    # membangun model word2vec untuk setiap kata yang muncul minimal 1 kali
    # by default, vector_size = 100 dan window_size = 5
    # vector_size = 100 berarti representasi suatu word akan sebanyak 100 vector angka
    word2vec = Word2Vec(data, min_count=1)
    
    # mengubah setiap word menjadi representasi vectornya
    data = [[word2vec.wv[word] for word in sentence] for sentence in data]
    # convert ke numpy array untuk akses shape
    data = np.array(data)
    # reshape ke bentuk flatten untuk input ke BPNN
    data = data.reshape(-1, data.shape[1]*data.shape[2])
    return data

# Input representasi word vector
x_vec = vectorize(x)
print("Vector representation:\n", x_vec)

# Menambah Number of Hashtags sebagai feature
x_vec = [np.append(vec, num_tag[i]) for i, vec in enumerate(x_vec)]
# convert ke numpy array untuk akses shape
x_vec = np.array(x_vec)
x_vec.shape

Vector representation:
 [[ 0.00242733 -0.00378142  0.00402846 ... -0.00027745  0.00397951
   0.00075789]
 [ 0.00122478 -0.00354308 -0.00489287 ... -0.00027745  0.00397951
   0.00075789]
 [-0.00408636  0.00057664 -0.00444802 ... -0.00345904  0.00466163
  -0.00051724]
 [-0.00335128  0.00338272  0.00468503 ... -0.00027745  0.00397951
   0.00075789]
 [ 0.00228935 -0.00158379 -0.00145577 ...  0.00390947 -0.00162096
  -0.0020406 ]
 [ 0.00331802 -0.00270061  0.00328912 ... -0.00027745  0.00397951
   0.00075789]]


(6, 1001)

## Split Data Train 4 data (2 Fear, 2 Joy) dan Test 2 data (1 Fear, 1 Joy)

In [6]:
train = [1, 2, 4, 5]
test = [0, 3]

x_train = np.array(x_vec[train])
y_train = np.array(y[train])
x_test = np.array(x_vec[test])
y_test = np.array(y[test])

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4, 1001), (4, 1), (2, 1001), (2, 1))

# C. Build Model BPNN
## a) Model BPNN
### 1. Architecture
#### - Input Layer (1001 neuron) 
#### - Hidden Layer 1 (500 neuron) 
#### - Hidden Layer 2 (100 neuron) 
#### - Hidden Layer 3 (25 neuron) 
#### - Output Layer (1 neuron)
### 2. Input dan Output
#### - Input menggunakan data word vector hasil word2vec yang sudah di flatten + number of hashtags dalam sentence
#### - Output menggunakan 0 dan 1 untuk representasi "fear" dan "joy"
### 3. Training Process
#### - Hidden Layer 1-3 tidak menggunakan activation function, agar hasil training tidak di compress dan nilai word similarity akan tetap terjaga dengan baik.
#### - Output Layer menggunakan Sigmoid agar hasil berada dalam range 0 sampai 1 sesuai Output yang diinginkan.
#### - Sigmoid juga dinyatakan bagus dalam binary classification.
#### - Selebihnya architecture merupakan hasil eksperimen / coba-coba untuk memperoleh model yang baik.

In [7]:
def create_model():
    model = Sequential()
    model.add(layers.Dense(500))
    model.add(layers.Dense(100))
    model.add(layers.Dense(25))
    model.add(layers.Dense(1, activation = "sigmoid"))
    model.compile("adam", "binary_crossentropy", "accuracy")
    return model

model = create_model()
model.fit(x_train, y_train, epochs = 20)

Epoch 1/20
1/1 [==============================] - 1s 780ms/step - loss: 0.6904 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6138 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 7ms/step - loss: 0.5817 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 6ms/step - loss: 0.5580 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 7ms/step - loss: 0.5264 - accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 8ms/step - loss: 0.4870 - accuracy: 0.5000
Epoch 7/20
1/1 [==============================] - 0s 11ms/step - loss: 0.4440 - accuracy: 0.7500
Epoch 8/20
1/1 [==============================] - 0s 8ms/step - loss: 0.4014 - accuracy: 0.7500
Epoch 9/20
1/1 [==============================] - 0s 7ms/step - loss: 0.3613 - accuracy: 1.0000
Epoch 10/20
1/1 [==============================] - 0s 7ms/step - loss: 0.3233 - accuracy: 1.0000
Epoch 11/20
1/1 [===================

# D. Performance of BPNN: accuracy, precision and recall

In [9]:
def performance(y_pred, y):
    # convert < 0.5 jadi 0
    # convert >= 0.5 jadi 1
    y_pred = y_pred.round()
    print("Confusion Matrix:")
    print(confusion_matrix(y, y_pred))
    print(f"Accuracy: {accuracy_score(y, y_pred)*100}%")
    print(f"Precision: {precision_score(y, y_pred)*100}%")
    print(f"Recall: {recall_score(y, y_pred)*100}%")

## Performance Training Data
### Seperti biasa, tentu saja training data akan mendapat nilai 100%. Hal ini terjadi karena data corpus terlalu sedikit sehingga model menjadi overfitting.

In [10]:
print("Training Data")
y_pred = model.predict(x_train)
performance(y_pred, y_train)

Training Data
Confusion Matrix:
[[2 0]
 [0 2]]
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%


## Performance Testing Data
### Dapat dilihat data testing tidak diprediksi dengan sempurna, hal ini menunjukkan overfitting kemungkinan besar terjadi.
### Selain itu, tidak semua corpus pada data test ada dalam corpus data train, sehingga juga mempengaruhi performa.

In [11]:
print("Testing Data")
y_pred = model.predict(x_test)
performance(y_pred, y_test)

Testing Data
Confusion Matrix:
[[0 1]
 [0 1]]
Accuracy: 50.0%
Precision: 50.0%
Recall: 100.0%


# E. Propose application
## Saya dapat mengumpulkan berita berhubungan dengan saham atau crypto dengan keyword tertentu dan melakukan analisa sentimentnya untuk mengetahui gambaran sentiment pasar saat ini.
## Dapat juga digunakan untuk mendapatkan gambaran review dari keseluruhan penilaian customer dari perusahaan sehingga tidak perlu cek satu per satu untuk mengetahui apakah pelayanan perusahaan tersebut sudah maksimal atau tidak.